In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentence_transformers
!pip install openpyxl
!pip install accelerate
!pip install bitsandbytes
!pip install  git+https://github.com/huggingface/peft.git
!pip install evaluate
# !pip install -v --disable-pip-version-check --no-cache-dir 'drive/MyDrive/DreamIdeaSoft data/codes/apex'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=68a2aa9b606273af42d8a90e335a5d796b7963487982eaef5beebf3863e9a9ec
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers
Looking in indexes: https://pypi.o

In [3]:
import torch

import pandas
import glob
import logging
import os
import pickle
import random
import re
import shutil
from tqdm import tqdm as tqdm
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

from transformers import AutoTokenizer, AutoModelForCausalLM 
from transformers import AutoModelWithLMHead, AutoModelForCausalLM, AutoTokenizer
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType, PeftModel, PeftConfig
from transformers import Trainer,TrainingArguments
from typing import Optional, Dict, Sequence

import transformers
import copy



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28

In [4]:
def data_prep_from_txt(file_name : str, o_data : dict = None) :
    '''
    Here we assume the data is a text file and follows the format :
    
    ■ location 
    
    
    [데이터]
    summary....
    
    [질의응답]
    Q1: question 1
    A1: answer1 
    .
    .
    .
    
    '''
    with open(file_name,'r') as fil :
        data=fil.read()
    data=data.replace('n\n\n\n\n\n\n\n\n\n','')
    data=data.split("■")
    #print(data)
    data=[data[i].split('[질의응답]') for i in range(len(data))]
    for i in range(len(data)) :
        try :
            data[i][1]=data[i][1].split('\nQ')
        except Exception as e  :
            continue
    for i in range(len(data)) :
        try :
            for j in range(len(data[i][1])) :
                data[i][1][j]=data[i][1][j].split('\nA')
        except Exception as e  :
            continue
    data=data[1:]
    #print(data)
    for i in range(len(data)) :
        data[i][1]=data[i][1][1:]
    
    dataf = {} if not o_data else o_data
    for i in range(len(data)):
        dataf[data[i][0].split('\n')[0][1:]] = {
               'context' : data[i][0].split('[데이터]')[1] +'\n',
               'question' : ['q : ' + data[i][1][j][0].split(':')[1].replace('\n','') +'\na : ' for j in range(len(data[i][1]))],
               'answer' : [data[i][1][j][1].split(':')[1].replace('\n','') +'\n' for j in range(len(data[i][1]))],
                }  
        
    return dataf

def make_contexted(data : dict,tokenizer) :
    '''
    Expects a dictionay of the form : 
        dict('location':{'context' : 'summary of the locations information', 
                         'question' : ['a list questions'],
                         'answer' : ['a list of answers']})
    '''
    contexted={'context' : [] ,'answer' : []}
    for j in data :
        l=[2047-tokenizer.encode(data[j]['answer'][i],return_tensors='pt').shape[1] for i in range(len(data[j]['answer']))]
        contexted['context'].extend([get_fitting({'context': data[j]['context'], 'question' : data[j]['question'][i]},l[i],tokenizer) for i in range(len(data[j]['answer']))])
        contexted['answer'].extend([data[j]['answer'][i] for i in range(len(data[j]['answer']))])
    contexted=pandas.DataFrame(contexted)
    #print(contexted)
    return contexted

def data_prep_from_excel(file_name : str, o_data : dict = None ) :
    '''
    Here we assume the data is in an excel file with the columns : No, Question, Answer
    '''
    location=file_name.split('/')[-1].split('.')[0]
    n_data=pd.read_excel(file_name)
    
    data = {} if not o_data else o_data
    
    if location in data : 
        data[location]['question'].extend([n_data['Question'][i] for i in range(len(n_data))])
        data[location]['answer'].extend([n_data['Answer'][i] for i in range(len(n_data))])
    
    
    else :
        data[location]={'context' : '' ,
                        'question' : [n_data['Question'][i] for i in range(len(n_data))],
                        'answer' : [n_data['Answer'][i] for i in range(len(n_data))]}
    
    return data
            
    

In [5]:
from sentence_transformers import SentenceTransformer, util
embedder = SentenceTransformer('jhgan/ko-sroberta-multitask')

def get_fitting(data,size,tokenizer) :
    hits=hits=util.semantic_search(embedder.encode(data['question']),embedder.encode(data['context'].split('\n\n')))
    x = data['context'] + data['question']
    l = len(hits[0])
    while tokenizer.encode(x,return_tensors="pt").shape[1] > size :
        l-=1
        x='\n\n'.join([data['context'].split('\n\n')[hits[0][i]['corpus_id']] for i in range(l,0,-1)])+ data['question']
    return x

def get_context(question : str, kD : dict) : 
    corpus = list(kD.keys())
    hits = util.semantic_search(embedder.encode(question),embedder.encode(corpus))
    return corpus(hits[0][0]['corpus_id']) if hits[0][0][''] > 0.7 else 'unknown' 

In [6]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

#### for new train
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    IGNORE_INDEX = -100
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_df, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logger.warning("Loading data...")

        logger.warning("Formatting inputs...")
        sources=[]
        targets=[]
        for _, row in data_df.iterrows():
            sources.append(row['context'])
            targets.append(row['answer'])
      
        logger.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""
    def __init__(self,tokenizer: transformers.PreTrainedTokenizer) :
        self.tokenizer=tokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        IGNORE_INDEX=-100
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

def make_supervised_data_module(tokenizer: transformers.PreTrainedTokenizer, data_df) -> Dict:
    train_dataset = SupervisedDataset(tokenizer=tokenizer, data_df=data_df)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    return dict(train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)

def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            # max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )


In [7]:
def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)



In [8]:
class Args():
    def __init__(self):
        self.output_dir = 'drive/MyDrive/DreamIdeaSoft data/codes/output-large'
        self.model_type = 'koGPt'
        self.model_name_or_path = 'kakaobrain/kogpt'
        self.config_name = 'kakaobrain/kogpt'
        self.tokenizer_name = 'kakaobrain/kogpt'
        self.cache_dir = 'drive/MyDrive/DreamIdeaSoft data/codes/cached'
        self.block_size = 2048 #tells the maximum number of tokens used per sample
        self.do_train = True #determines whether the model will be trained or not
        self.do_eval = False #determines whether the model will be evaluated or not
        self.evaluate_during_training = False #determine wether the model is evaluated during training or not. this can help you assess the generalizaation ability of the model during the training
        self.per_gpu_train_batch_size = 4 #determines the number of samples per training batch
        self.per_gpu_eval_batch_size = 1 #determines the number of samples per evaluation batch
        self.gradient_accumulation_steps = 10 #determines how_many steps over whhich the gradient is accumulated before performing back propagation and updating the weights. accumulating over more steps helps reduce the gpu memory usage during training but might make the training slower
        self.gradient_checkpointing = True #this makes the model save some of the activations during the forward pass in order to avoid recomputing them during the back propagation. helps reduce the gpu memory usage during training
        self.learning_rate = 5e-5 #sets the initial learning rate of the optimizer
        self.weight_decay = 0.0 #can help prevent overfiting and keep the weights low. it is a kind of regularizer
        self.adam_epsilon = 1e-8 #one of the hyper parameters of the optimizer. we are using adamw optimizer because it is the most comonly used for nlp(more information on adamw https://openreview.net/pdf?id=Bkg6RiCqY7, if you want to checkout other optimizers you can go here https://paperswithcode.com/methods/category/optimization)
        self.max_grad_norm = 1.0 #defines the maximum value of the l2 norm of the gradient. if it is greater than this the gradient will be reduced to make its norm be equal to this. this helps prevent exploding gradients which might lead to the model not learning
        self.num_train_epochs = 10 #defines the number of training epochs
        self.max_steps = -1 #sets the maximum number of steps per training epoch (-1 means there is no limit)
        self.warmup_steps = 0 #defines the number of warmup steps, using warmup can help tune the attention mechanism of the transformer to the data, might help improve the performance of the model
        self.logging_steps = 500 #the number of steps after which the training states are saved
        self.save_steps = 0 #number of steps after which the model is trained during the training
        self.save_total_limit = None #total number times the model should be saved during training
        self.eval_all_checkpoints = False #determines whether all checkpoints should be evaluated
        self.no_cuda = True 
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False #determines wether to use mixed precision training(more info https://arxiv.org/abs/1710.03740), this can speedup the training process but will require aditional libraries to be installed
        self.fp16_opt_level = 'O1'

args = Args()

In [9]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )
    if args.gradient_checkpointing :
      model.gradient_checkpointing_enable()
    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        torch.cuda.empty_cache()
        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue
            torch.cuda.empty_cache()
            inputs, labels = (batch, batch)
            if inputs.shape[1] > 2048: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
            outputs=''
            inputs=''
            labels=''
            torch.cuda.empty_cache()
            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            loss=''
            torch.cuda.empty_cache()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)
                torch.cuda.empty_cache()
                     
            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result
    
# Main runner

def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    #config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained( 'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
              bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]')
   
    model = AutoModelForCausalLM .from_pretrained(
        args.model_name_or_path,
        revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
        pad_token_id=tokenizer.eos_token_id,
        torch_dtype='auto', low_cpu_mem_usage=True
       ).to(device='cuda', non_blocking=True)

    # in case you already have a pretrained model in your drive you can replace the path below with the path to the model
    # tokenizer = AutoTokenizer.from_pretrained('drive/MyDrive/DreamIdeaSoft data/codes/kogpt')
    # model = AutoModelForCausalLM .from_pretrained('drive/MyDrive/DreamIdeaSoft data/codes/kogpt').to(device='cuda', non_blocking=True)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)
    input()
    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))
        torch.cuda.empty_cache()
        #Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelForCausalLM.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        torch.cuda.empty_cache()
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelForCausalLM.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results



In [10]:
import evaluate as ev
def train(args,train_dataset,model,tokenizer):
   
    data_module = make_supervised_data_module(tokenizer=tokenizer, data_df=train_dataset)
    training_args=TrainingArguments(output_dir=args.output_dir,
                                    gradient_accumulation_steps = args.gradient_accumulation_steps,
                                    weight_decay=args.weight_decay,
                                    learning_rate=args.learning_rate,
                                    adam_epsilon=args.adam_epsilon,
                                    max_grad_norm=args.max_grad_norm,
                                    warmup_steps=args.warmup_steps,
                                    max_steps=args.max_steps,
                                    gradient_checkpointing = args.gradient_checkpointing,
                                    per_device_train_batch_size=args.per_gpu_train_batch_size,
                                    optim='adamw_bnb_8bit',

                                    fp16= args.fp16,
                                    num_train_epochs = args.num_train_epochs)
     
    trainer = Trainer(model=model, tokenizer=tokenizer, args=training_args, **data_module)
    trainer.train()
    trainer.model.save_pretrained(args.output_dir)
    trainer.tokenizer.save_pretrained(args.output_dir)
    #trainer.save_state()
    #safe_save_model_for_hf_trainer(trainer=trainer, output_dir='drive/MyDrive/DreamIdeaSoft data/codes/output-large')

def evaluate_Exact_match(args,eval_dataset,model,tokenizer):
    data_module = make_supervised_data_module(tokenizer=tokenizer, data_df=eval_dataset)
    predictions=[]
    labels=[]
    for point in data_module['train_dataset'] :
        k=[j for j in point['labels'] if j == -100]
        labels.append(tokenizer.decode(point['labels'][len(k):],skip_special_tokens=True))
        model_inp=tokenizer.decode(point['input_ids'],skip_special_tokens=True)
        model_inp=tokenizer.encode(model_inp,return_tensors='pt').to(model.device)
        gen_toks=model.generate(input_ids=model_inp,
                                max_new_tokens=2048-model_inp.shape[-1],
                                num_return_sequences=1,
                                temperature=0.5,
                                no_repeat_ngram_size=6,
                                do_sample=True)
        predictions.append(tokenizer.batch_decode(gen_toks,skip_special_tokens=True)[0])
    metric=ev.load('exact_match')
    return {'exact_match' : metric.compute(references=labels,predictions=predictions)}

def peft_model(model_name,configs) :
    model = AutoModelForCausalLM .from_pretrained(model_name, low_cpu_mem_usage=True,device_map='auto',load_in_8bit=True,**configs)
    lora_config = LoraConfig(r=8, lora_alpha=32, lora_dropout=0.1, task_type=TaskType.CAUSAL_LM)
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, lora_config)
    return model


def do_train(df_trn, df_val,model=None,tokenizer=None):
    
    IGNORE_INDEX = -100
    device = torch.device("cuda")
    args= Args()
    args.n_gpu = torch.cuda.device_count()
    args.device=device
    set_seed(args)

    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        try :
  
            checkpoint=args.output_dir
            config=PeftConfig.from_pretrained(checkpoint)
            tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
            model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,device_map='auto',load_in_8bit=True)
            model = PeftModel.from_pretrained(model, checkpoint, device_map={"":0})

        except Exception as e :
            tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path,'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
                  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]')
            configs=dict(revision='KoGPT6B-ryan1.5b-float16', pad_token_id=tokenizer.eos_token_id, torch_dtype='auto')
            model=peft_model(args.model_name_or_path,configs)      
            train(args, df_trn, model, tokenizer)
        
  
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))
        torch.cuda.empty_cache()
        

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]  
        checkpoint=args.output_dir
        config=PeftConfig.from_pretrained(checkpoint)
        tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
        model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,device_map='auto',load_in_8bit=True)
        model = PeftModel.from_pretrained(model, checkpoint, device_map={"":0})
        results = evaluate(args, model, tokenizer, df_trn, df_val, prefix="")
            #

    return results



In [11]:
df={}
tokenizer = AutoTokenizer.from_pretrained( 'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
              bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]')
path_dir='drive/MyDrive/DreamIdeaSoft data/' #replace with the path to the txt data files
for fil in os.listdir(path_dir) :
  if not fil.endswith('txt') :
    continue
  df=data_prep_from_txt(path_dir+'/'+fil,df)
dt=make_contexted(df,tokenizer)

In [12]:
trn_df, val_df = train_test_split(dt, test_size=0.1)

In [13]:
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [14]:
### the koGPT model is an autoregressive model based on GPT-J(https://github.com/kingoflolz/mesh-transformer-jax , https://huggingface.co/EleutherAI/gpt-j-6b) which uses the transforemer architecture(https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf).
#main(trn_df, val_df)

### if you have a cuda out of  memory issue instead do it loads the model where parts of the model are processed in int8 rather than float16 or float32 to reduce the space usage but might reduce in a drop in performance(more info at https://arxiv.org/pdf/2208.07339.pdf) of the model also use LoRA training where only a part of the model is trained to further reduce space consumption (you can check https://arxiv.org/pdf/2106.09685.pdf for more information LoRa)
do_train(trn_df,val_df)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

{}

In [15]:
args=Args()
config=PeftConfig.from_pretrained(args.output_dir)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,device_map='auto',load_in_8bit=True)
model = PeftModel.from_pretrained(model, 'drive/MyDrive/DreamIdeaSoft data/codes/output-large', device_map={"":0})


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
while (1):
  question=input('>>')
  if question=='bye':
      break;
  

# for i in range(len(dt)) :
  # if len(dt['context'][i]) > 3000:
  #   continue
  # input_ids=tokenizer.encode(dt['context'][i],return_tensors="pt").to(model.device)
  context= df['팔공산 갓바위']['context']
  
  x=get_fitting({'context':context,'question':question},2048-128,tokenizer)
  input_ids=tokenizer.encode(x,return_tensors='pt').to('cuda')
  output_toks=model.generate(input_ids=input_ids,
                 max_new_tokens=128,
                 num_return_sequences=1,
                 temperature=0.5,
                 no_repeat_ngram_size=6,
                 do_sample=True,
               )
  print('bot>>',tokenizer.batch_decode(output_toks[:, input_ids.shape[1]:][0],skip_special_tokens=True)[0])


>>안녕
bot>> 하
>>팔공산 가는 길
bot>> 을
>>팔공산 가는 길 알려줘
bot>>  있
>>bye
